# Algorithm 26: Rename Symmetric Ground Truth Atoms

Some amino acids have chemically equivalent atoms that can be swapped (e.g., the two oxygens of aspartate). This algorithm finds the optimal naming to minimize loss.

## Algorithm Pseudocode

![renameSymmetricGroundTruthAtoms](../imgs/algorithms/renameSymmetricGroundTruthAtoms.png)

## Source Code Location
- **File**: `AF2-source-code/model/folding.py`
- **Function**: `compute_renamed_ground_truth`
- **Lines**: 428-480

## Symmetric Amino Acids

| Amino Acid | Symmetric Atoms |
|------------|----------------|
| Asp (D) | OD1 ↔ OD2 |
| Glu (E) | OE1 ↔ OE2 |
| Phe (F) | CD1 ↔ CD2, CE1 ↔ CE2 |
| Tyr (Y) | CD1 ↔ CD2, CE1 ↔ CE2 |
| Arg (R) | NH1 ↔ NH2 |

In [ ]:
import numpy as np

np.random.seed(42)

In [ ]:
def rename_symmetric_ground_truth_atoms(pred_pos, gt_pos, aatype, symmetric_atoms):
    """
    Rename Symmetric Ground Truth Atoms - Algorithm 26.
    
    Finds optimal atom naming for symmetric residues.
    
    Args:
        pred_pos: Predicted atom positions [N_res, N_atoms, 3]
        gt_pos: Ground truth atom positions [N_res, N_atoms, 3]
        aatype: Amino acid types [N_res]
        symmetric_atoms: Dict mapping aa_type to list of (atom1, atom2) swap pairs
    
    Returns:
        gt_pos_renamed: Renamed ground truth positions
    """
    N_res, N_atoms, _ = gt_pos.shape
    gt_renamed = gt_pos.copy()
    
    print(f"Rename Symmetric Ground Truth Atoms")
    print(f"  Residues: {N_res}")
    print(f"  Atoms per residue: {N_atoms}")
    
    n_swapped = 0
    
    for res_idx in range(N_res):
        aa = aatype[res_idx]
        
        if aa not in symmetric_atoms:
            continue
        
        swap_pairs = symmetric_atoms[aa]
        
        # Compute loss for original naming
        loss_original = 0
        for a1, a2 in swap_pairs:
            loss_original += np.sum((pred_pos[res_idx, a1] - gt_pos[res_idx, a1])**2)
            loss_original += np.sum((pred_pos[res_idx, a2] - gt_pos[res_idx, a2])**2)
        
        # Compute loss for swapped naming
        loss_swapped = 0
        for a1, a2 in swap_pairs:
            loss_swapped += np.sum((pred_pos[res_idx, a1] - gt_pos[res_idx, a2])**2)
            loss_swapped += np.sum((pred_pos[res_idx, a2] - gt_pos[res_idx, a1])**2)
        
        # Swap if it reduces loss
        if loss_swapped < loss_original:
            for a1, a2 in swap_pairs:
                gt_renamed[res_idx, a1] = gt_pos[res_idx, a2].copy()
                gt_renamed[res_idx, a2] = gt_pos[res_idx, a1].copy()
            n_swapped += 1
    
    print(f"  Swapped: {n_swapped} residues")
    
    return gt_renamed

In [ ]:
# Test
N_res, N_atoms = 10, 14

# Predicted positions
pred_pos = np.random.randn(N_res, N_atoms, 3)

# Ground truth - some atoms swapped
gt_pos = pred_pos.copy() + np.random.randn(N_res, N_atoms, 3) * 0.5

# Swap atoms 5 and 6 for some residues in ground truth
swap_indices = [0, 2, 5]
for idx in swap_indices:
    gt_pos[idx, 5], gt_pos[idx, 6] = gt_pos[idx, 6].copy(), gt_pos[idx, 5].copy()

# Amino acid types (0 = symmetric type)
aatype = np.array([0, 1, 0, 1, 1, 0, 1, 1, 1, 1])

# Define symmetric atoms for aa_type 0
symmetric_atoms = {
    0: [(5, 6)],  # Atoms 5 and 6 are symmetric
}

print("Test Rename Symmetric Ground Truth Atoms")
print("="*50)

gt_renamed = rename_symmetric_ground_truth_atoms(pred_pos, gt_pos, aatype, symmetric_atoms)

# Verify loss reduction
print(f"\nVerification:")
loss_before = np.sum((pred_pos - gt_pos)**2)
loss_after = np.sum((pred_pos - gt_renamed)**2)
print(f"  Loss before: {loss_before:.2f}")
print(f"  Loss after: {loss_after:.2f}")
print(f"  Improvement: {loss_before - loss_after:.2f}")

## Source Code Reference

```python
# From AF2-source-code/model/folding.py

def compute_renamed_ground_truth(batch, atom14_pred_positions):
  """Find optimal renaming for symmetric atoms.

  Jumper et al. (2021) Suppl. Alg. 26
  """
  # Compute chi angles from predicted positions
  # Compare with ground truth chi angles
  # Find optimal alt_naming_is_better mask
  # Swap atoms accordingly
  ...
```